In [0]:
from pyspark.sql.functions import col

# =========================
# Lire les tables Silver
# =========================
erp_df = spark.table("lakehouse.silver.erp_custt")
loc_df = spark.table("lakehouse.silver.erp_locationn")

# ======================================
# Jointure ERP customer + location
# ======================================
erp_full_df = erp_df.join(
    loc_df,
    erp_df["CUST_ID"] == loc_df["CUST_ID"],
    "left"
)

# =========================
# Sélection finale Gold
# =========================
gold_dim_erp_cust_df = erp_full_df.select(
    erp_df["Customer_id"],
    erp_df["CUST_ID"],
    col("Customer_Gender"),
    col("Date_birth"),
    erp_df["CID"],
    col("Country")
)

# =========================
# Écriture Gold layer
# =========================
gold_dim_erp_cust_df.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("lakehouse.gold.dim_customerss")

print("Gold dimension dim_customerss created successfully")


In [0]:
from pyspark.sql.functions import col

# =========================
# Lire les tables Silver
# =========================
cust_df = spark.table("lakehouse.gold.dim_customerss")
crm_cust_df = spark.table("lakehouse.silver.crm_cust_info")

# ======================================
# Jointure ERP customer + location
# ======================================
gold_crm_cust_df = crm_cust_df.join(
    cust_df,
    crm_cust_df["cst_key"] == cust_df["CUST_ID"],
    "left"
)

# =========================
# Sélection finale Gold
# =========================
gold_dim_cust_df = gold_crm_cust_df.select(
    cust_df["Customer_id"],
    cust_df["CUST_ID"],
    crm_cust_df["cst_id"],
    cust_df["Customer_Gender"],
    cust_df["Date_birth"],
    cust_df["CID"],
    cust_df["Country"],
    crm_cust_df["cst_firstname"],
    crm_cust_df["cst_lastname"],
    crm_cust_df["cst_marital_status"],
    crm_cust_df["cst_gndr"],
    crm_cust_df["cst_create_date"]


)

# =========================
# Écriture Gold layer
# =========================
gold_dim_cust_df.write \
    .mode("overwrite") \
    .format("delta") \
    .saveAsTable("lakehouse.gold.dim_customers")

print("Gold dimension dim_customerss created successfully")
